Importing libraries

In [500]:
import os
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import base64
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.hashes import SHA256
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [501]:
df = pd.read_csv('D:/DPS Lab/ILLNESS-INSIGHT-main/dataset/parkinsons.csv')

In [502]:
df.columns

Index(['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [503]:
data=df.drop(['status','name'],axis=1)
y=df['status']

In [504]:
# Function to generate a secure key using PBKDF2
def generate_key(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=SHA256(),
        length=32,
        salt=salt,
        iterations=100000,  # You can adjust the number of iterations based on your security requirements
        backend=default_backend()
    )
    return kdf.derive(password)

# Encrypt using AES algorithm in CFB mode
def encrypt_df(df, key, iv):
    backend = default_backend()
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=backend)

    encrypted_data = {}
    for col in df.columns:
        encrypted_col = []
        for value in df[col]:
            encryptor = cipher.encryptor()
            # Convert the value to bytes
            value_bytes = str(value).encode('utf-8')
            # Encrypt the data
            encrypted_value = encryptor.update(value_bytes) + encryptor.finalize()
            encrypted_col.append(base64.b64encode(encrypted_value).decode())

        encrypted_data[col] = encrypted_col

    encrypted_df = pd.DataFrame(encrypted_data)
    return encrypted_df


# Convert base64-encoded DataFrame to integer
def base64_to_int(df):
    return df.applymap(lambda x: int.from_bytes(base64.b64decode(x.encode()), byteorder='big'))

# Example password and salt (for demonstration purposes only)
password = b'SecurePassword123'
salt = os.urandom(16)
print(salt)
# Generate a secure key
key = generate_key(password, salt)

# Generate a random IV (Initialization Vector)
iv = os.urandom(16)
print(iv)
# Encrypt the DataFrame
data_encrypted = encrypt_df(data, key, iv)

# Convert DataFrame to base64
data_base64 = data_encrypted.applymap(lambda x: base64.b64encode(x.encode()).decode())

# Convert base64-encoded DataFrame to integer
data_final = base64_to_int(data_base64)

b'\xf1\xcacC*\xf3\xa6\x90\x8bV`h\xd6H\x83\xb3'
b'\xb4\x8e\x91\x96<-\xf2\xa4\xdcc\xfb\xbc\x82\xb3r\x16'


C:\Users\manik\AppData\Local\Temp\ipykernel_7844\1706818423.py:52: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_base64 = data_encrypted.applymap(lambda x: base64.b64encode(x.encode()).decode())
C:\Users\manik\AppData\Local\Temp\ipykernel_7844\1706818423.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: int.from_bytes(base64.b64decode(x.encode()), byteorder='big'))


In [505]:
X_train, X_test, Y_train, Y_test = train_test_split(data_final, y, test_size=0.2, random_state=2)

In [506]:
print(data_final.shape, X_train.shape, X_test.shape)

(195, 22) (156, 22) (39, 22)


Model Training

Support Vector Machine Model

In [507]:
model = svm.SVC(kernel='linear')

In [508]:
pd.set_option('display.max_columns', None)

In [509]:
X_train

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
123,23932283707733286006368451901,23932303888471307033854950717,5654927331512042550,23911694152747857055018204477,5645099883694207037,23911694152747838380567510333,23911694152747916428867747133,23911694152764445399899520317,23911694152758252954639744317,5567375149030075197,23911694152764519058520489277,5567375140442616625,23911694152764468433744510269,23911694152819960841500048701,23911694152764464091530083645,5572169290326885429,23911695443875831403248694333,23911695480919391892679717693,23622922641430860004691375933,23911694134373221152948892733,23957633315524496789177381949,23911694189657933530136658237
160,23932129142464701134293908797,23932148031930637032176565565,5654960316857020975,23911694152747838346144202045,5645099883694207037,23911694152747907611299888445,23911694152747912043551604029,23911694152747857033610476861,23911694152819956447899499837,5567375445381428285,23911694152764445391463071037,23911694152764468455285407037,23911694152819960841499000125,23911694152819891516429385021,23911694152758313414894370109,5578403521255011894,23911695499448894041946998077,23911695499232725640766974781,23622922641375350060869567538,23911694189730014192378201405,23957633315535179699984952125,23911694189713462148698227005
94,23932407780533883956499004733,23932444268193666182170492221,5650419333851081272,23911694152747912026525220157,5579234739143913533,23911694152747903213098843453,23911694152747903213098843453,23911694152747838346208820541,23911694152753744952736955709,5567375144733718589,23911694152758313419188288829,23911694152758326578971622717,23911694152758248548071849277,23911694152830375385573571901,5567375140438754356,5578329853961596464,23911695444019992749492616509,23911695480930087976219786557,23622941530835070415297868336,23911694134156997810478724157,23957633315518581442453007165,23911694134383626913745825597
57,23932129160911445255097105725,23932146851338965771656183101,23932127832745880756917714237,23911694152747833960979053885,5651292333181841469,23911694152747907611299888445,23911694152747916437457681725,23911694152747857033763962173,23911694152758322185217588541,5567375149031123773,23911694152764510241174273341,23911694152764523448198708541,23911694152764468476911238461,23911694152753749346641984829,23911694152747842744190713149,5578329853925617206,23911695444025894966344959293,23911694300338393582937388093,23622903751975325495179228221,23911694189513878836609447741,23957633333903673713205799741,23911694171211267526049036093
41,23932284980558627074997697853,23932303906917986189200932157,23932426688446563936965311805,23911694152747907628478709053,5573042289656279101,23911694152747903191626497341,23911694152747903234574728509,23911694152747916445827022141,23911694152764523460863016253,5567375136145174333,23911694152747833965274021181,23911694152747861470248123709,23911694152764514613298937149,23911694152758322180988681533,23911694152747907628326272317,5578394725161972523,23911694171194586835146535741,23911695481002099352049316669,23622960420290665372110121520,23911694152830384198774779709,23957633352422475380737601341,23911694134317701317532925245
66,23932387562902434172181626173,23932406581495496105764797757,5650456717229650488,23911694152747857067971263805,5651292333181841469,5567375140438768171,5567375140438768181,23911694152764514639220784445,23911694152753814187763252541,5567375144737600317,5567375140442620971,5567375140442620984,23911694152758330977235189053,5567375140459412016,23911694152758317791466044733,5572173688375882813,23911695462549200250900727101,23911695480923886696214053181,23622922641430842395396436601,23911694189651745465815219517,23957633352428626031670620989,23911694189651745495875400765
98,23932148013483814893143932221,23932387562902360500672216381,5716317463735855671,2391169415

In [510]:
X_train.columns

Index(['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1',
       'spread2', 'D2', 'PPE'],
      dtype='object')

In [511]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


In [512]:
from sklearn.svm import SVC

model = SVC(C=1.0)  # You can adjust the value of C
model.fit(X_train_scaled, Y_train)


SVC()

Model Evaluation

Accuracy Score

In [513]:
# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

C:\Users\manik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [514]:
print('Accuracy score of training data : ', training_data_accuracy)

Accuracy score of training data :  0.7435897435897436


In [515]:
# accuracy score on training data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

C:\Users\manik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [516]:
print('Accuracy score of test data : ', test_data_accuracy)

Accuracy score of test data :  0.7948717948717948


Building a Predictive System

In [517]:
input_data = (37547310455488283079774125373,37547329400294451168977304893,8824922647968510036,37526517912381968845411663165,8815046838818136893,37526517912381955668468776253,37526517912381944656120855869,37526517912401940396111248701,37526517912397436792172133693,8737323311040580925,37526517912401907402154655037,8737323319633011811,37526517912401957996867960125,37526517912309873889468562749,37526517912401953590234004797,8742164606749798481,37526517967866320734282807101,37526517930837951097420081981,15264305372496235871195313213,37526517893863116600146491197,37572457075017787601341347645,37526517875508452231510182205)

# changing input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the numpy array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = model.predict(input_data_reshaped)
print(prediction)


if (prediction[0] == 0):
  print("The Person does not have Parkinsons Disease")

else:
  print("The Person has Parkinsons")


[1]
The Person has Parkinsons


In [518]:
input_data = ()
# changing input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the numpy array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = model.predict(input_data_reshaped)
print(prediction)


if (prediction[0] == 0):
  print("The Person does not have Parkinsons Disease")

else:
  print("The Person has Parkinsons")

ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required by SVC.

Saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'parkinsons_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
filename = 'parkinsons_scalar.pkl'
pickle.dump(scaler, open(filename, 'wb'))

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('parkinsons_model.sav', 'rb'))

C:\Users\manik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.3.0 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
